In [1]:
from ipynb.fs.full.UtilitiesConceptDrift import *

In [2]:
test_set = OpenDataset("FullDataset.csv")
print(f"Number of tweets: {len(test_set)}")

Number of tweets: 33602


In [25]:
test_set = test_set.sort_values(by=['likes_count'], ascending=False)

In [30]:
data = search_tweet_Period(test_set, "2021-02-01", "2021-02-28").copy()

In [31]:
data.to_csv("ConceptDriftFebbraio.csv")

In [ ]:
#Do classification by hand

In [6]:
#Reload data
data = pd.read_csv("ConceptDriftFebbraio.csv",sep=',',usecols=['timestamp','username','tweet','label'])
data=data.dropna()
training_set = pd.read_csv("training_set.csv",sep=',',usecols=['timestamp','username','tweet','label'])

In [7]:
#Merge the all the data
new_trainin_set = pd.concat([training_set,data])
new_trainin_set.label=new_trainin_set.label.astype(int)

In [3]:
#print(new_trainin_set)
#print(f"Number of tweets: {len(new_trainin_set)}")

In [9]:
#Training the model
clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=3, analyzer="word", stop_words = set(stopwords.words('italian')), ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf',BernoulliNB()),
])
clf.fit(new_trainin_set.tweet, new_trainin_set.label)

Pipeline(steps=[('vect',
                 StemmedCountVectorizer(min_df=3,
                                        stop_words={'a', 'abbia', 'abbiamo',
                                                    'abbiano', 'abbiate', 'ad',
                                                    'agl', 'agli', 'ai', 'al',
                                                    'all', 'alla', 'alle',
                                                    'allo', 'anche', 'avemmo',
                                                    'avendo', 'avesse',
                                                    'avessero', 'avessi',
                                                    'avessimo', 'aveste',
                                                    'avesti', 'avete', 'aveva',
                                                    'avevamo', 'avevano',
                                                    'avevate', 'avevi', 'avevo', ...})),
                ('tfidf', TfidfTransformer()), ('clf', BernoulliNB())])

In [10]:
#Save the new model
joblib.dump(clf, 'modelFebbraio.pkl', compress=9)

['modelFebbraio.pkl']

In [11]:
#Load next month tweet classified
test_set = pd.read_csv("ConceptDriftMarzo.csv",sep=',',usecols=['timestamp','username','tweet','label'])
test_set = test_set.dropna()
test_set.label=test_set.label.astype(int)

In [29]:
#print(test_set)
#print(f"Number of tweets: {len(test_set)}")

In [13]:
#Do classification with new model
test_set["predicted"]= clf.predict(test_set.tweet)

In [30]:
#print(test_set)

In [27]:
#Benchmark new model
benchmark(test_set["label"], test_set["predicted"])

accuracy : 0.45
f_score : 0.4425837320574162

[[0.35 0.2  0.45]
 [0.25 0.65 0.1 ]
 [0.3  0.35 0.35]]
Precision,Recall,f_score
 Classe 0:  [0.38888889 0.54166667 0.38888889] 
 Classe 1:  [0.35 0.65 0.35] 
 Classe 2:  [0.36842105 0.59090909 0.36842105] 
 Support:   [20 20 20]


In [17]:
#Load old model
clf2 = joblib.load('InitialModel.pkl')

In [18]:
#Do classification with old model
test_set["predicted_old"]= clf2.predict(test_set.tweet)

In [26]:
test_set.predicted_old=test_set.label.astype(int)
#print(test_set)

In [25]:
benchmark(test_set["label"], test_set["predicted_old"])

accuracy : 1.0
f_score : 1.0

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Precision,Recall,f_score
 Classe 0:  [1. 1. 1.] 
 Classe 1:  [1. 1. 1.] 
 Classe 2:  [1. 1. 1.] 
 Support:   [20 20 20]
